# SK get car sales descriptions in English and Spanish

- Get a list of cars and for each car get a sales descriptions in English and translate it into Spanish.

## Load the required .NET packages and supporting classes

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 0.17.230626.1-preview"
#r "nuget: Microsoft.Extensions.Logging.Console"
#r "nuget: dotenv.net"

using System.ComponentModel;
using System.Net.Http;
using System.Text.Json;
using Microsoft.Extensions.Logging;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Skills.Core;
using Microsoft.SemanticKernel.SkillDefinition;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.AI.ImageGeneration;
using Microsoft.SemanticKernel.AI.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI.ChatCompletion;
using Microsoft.SemanticKernel.Planning;
using Microsoft.SemanticKernel.Planning.Sequential;

using dotenv.net;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

#!import Utils/ConsoleLogger.cs

// Json Options
readonly JsonSerializerOptions s_jsonOptions = new() { WriteIndented = true };

// Models
record Car(string VIN, string Make, string Model, int Year, string Color, string Motor, string Package, int milage, double price);
record CarTranslation(string VIN, string En, string ES);

## Load the OpenAI variables from environment variables or an .env file

In [ ]:
DotEnv.Load();
var deploymentName = Environment.GetEnvironmentVariable("DAVINCI_OPENAI_DEPLOYMENT_NAME");
var endpoint = Environment.GetEnvironmentVariable("DAVINCI_OPENAI_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("DAVINCI_OPENAI_KEY");
Console.WriteLine($"Using deployment: {deploymentName} at: {endpoint} with key {apiKey.Substring(0, 5)}...");

## Mock a database call to get car descriptions

In [ ]:
List<Car> MockDBCall() {
  var cars = new List<Car>{
    new Car("123","Ford", "Explorer", 2015, "Silver", "V6", "Platinum", 10000, 16500),
    new Car("456","Ford", "Mustang", 2018, "Blue", "V8", "Sports", 10000, 250000)
    //,    new Car("789","Ford", "Escape", 2020, "Red", "V6","Special", 3000, 15000)
  };
  return cars;
}

## Get a a short car description from the car properties

In [ ]:
string GetCarDescription(Car car) {
  return $"{car.Year} {car.Make} {car.Model} {car.Color} {car.Motor} {car.Package} with {car.milage} miles for ${car.price}";
}

## Get an instance of a kernel

In [ ]:
var kernel = new KernelBuilder()
    .WithLogger(ConsoleLogger.Log)
    .WithAzureTextCompletionService(deploymentName,endpoint, apiKey)
    .Build();

## Create the sales definition and translation in-line functions

In [ ]:
const string fnSalesDefinition = @"Get a used car sales description for the following: {{$input}}.";

var salesDescriptionFunc = kernel.CreateSemanticFunction(fnSalesDefinition, maxTokens: 200, temperature: 0.5, topP: 1);

const string fnESTranslatorDefinition = @"Translate the following English text to Spanish: {{$input}}.";

var esTranslatorFunc = kernel.CreateSemanticFunction(fnESTranslatorDefinition, maxTokens: 200, temperature: 0.5, topP: 1);

## Get a sales description and translate into Spanish

In [ ]:
// Mock making a call to a database to get a list of cars
var cars = MockDBCall();

var processedCars = new List<CarTranslation>();
foreach(var car in cars) {

  // Get a short description given the car properties
  var desc = GetCarDescription(car);
  Console.WriteLine($"Processing: {desc}");
  
  // Get a sales description from Davinci
  SKContext result = await kernel.RunAsync(desc,salesDescriptionFunc);
  Console.WriteLine($"Sales Description: {result}");

  // Get the usage for the call
  Console.WriteLine(JsonSerializer.Serialize(result.ModelResults.LastOrDefault()?.GetOpenAITextResult()?.Usage, s_jsonOptions));
  
  // Translate into Spanish
  SKContext es = await kernel.RunAsync(result.ToString(),esTranslatorFunc);
  Console.WriteLine($"Translation: {es}");
  
  // Add processed car to list
  processedCars.Add(new CarTranslation(car.VIN, result.ToString(), es.ToString()));
}

// Show the processed cars
processedCars